In [ ]:
import numpy as np

# Word Embeddings

The way machine learning models "see" data is different from how we (humans) do. For example, we can easily understand the text "I saw a cat", but our models can not - they need vectors of features. Such vectors, or word embeddings, are representations of words which can be fed into your model.

In practice, you have a vocabulary of allowed words; you choose this vocabulary in advance. For each vocabulary word, a look-up table contains its embedding. This embedding can be found using the word index in the vocabulary (i.e., you to look up the embedding in the table using word index).


To account for unknown words (the ones which are not in the vocabulary), usually a vocabulary contains a special token "UNK". Alternatively, unknown tokens can be ignored or assigned a zero vector.

The main objective of this week is: how do we get these word vectors?

# Representation as Discrete Symbols

### One hot vectors

The most simplest way of representing words in numerical data is to represent them as one hot vectors i.e for the ith word in the vocabulary, the vector has 1 on the ith dimension and 0 on the rest. This is also the most simplest way of represting categorical data in machine learning

In [5]:
vocab = ["the", "cat", "is", "dog", "grumpy", "nice", "sweet", "table", "corn","<UNK>"]

def convert_to_one_hot_vector(sentence):
    vectors = []
    for word in sentence.split(" "):
        vector = [0] * len(vocab)
        try:
            idx = vocab.index(word)
        except:
            idx = len(vocab) - 1
        vector[idx] = 1
        vectors.append(vector)
    return vectors

sentence = "the cat is very grumpy"
embeddings = convert_to_one_hot_vector(sentence)

print(sentence)
embeddings

the cat is very grumpy


[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 1, 0, 0, 0, 0, 0]]

Now the reason one hot vectors are not very popular in representing words is because for very large vocabularies, these vectors become very long and end up taking redundant space, vector dimensionality is equal to the vocabulary size

But the worst part about them is that these vectors know nothing about the words they represent. As you can see "the" is as close to "cat" than to "is". They do not capture the underlying meaning

How do we represent words so as to capture meaning?

### Distributional Semantics

To represent meaning let us first define the notion of what "meaning" is that we want to represent

consider the sentence: "the ___ is very grumpy"

What all words does your brain think can fit into the context of this sentence? Maybe a "cat" or a "dog" but definitely not a "table" 

The hypothesis is that your brain searched for other words that can be used in the same contexts, found some (e.g., dog, cat), and made a conclusion that cat has meaning similar to those other words(They both are grumpy). This is the distributional hypothesis:

Words which frequently appear in similar contexts have similar meaning.

Often you can find it formulated as "You shall know a word by the company it keeps" with the reference to J. R. Firth in 1957, but actually there were a lot more people responsible, and much earlier. For example, Harris, 1954.

This is an extremely valuable idea: it can be used in practice to make word vectors capture their meaning. According to the distributional hypothesis, "to capture meaning" and "to capture contexts" are inherently the same. Therefore, all we need to do is to put information about word contexts into word representation.

Main idea: We need to put information about word contexts into word representation.

All we'll be doing this week is looking at different ways to do this.

# Count Based Methods

So our main idea here is: We have to put information about contexts into word vectors.

Count-based methods take this idea quite literally, Put this information manually based on global corpus statistics.

The general procedure consists of the two steps:
(1) construct a word-context matrix,
(2) reduce its dimensionality.
There are two reasons to reduce dimensionality. First, a raw matrix is very large. Second, since a lot of words appear in only a few of possible contexts, this matrix potentially has a lot of uninformative elements (e.g., zeros).

To estimate similarity between words/contexts, usually you need to evaluate the dot-product of normalized word/context vectors (i.e., cosine similarity).

To define a count-based method, we need to define two things:
possible contexts (including what does it mean that a word appears in a context),
the notion of association, i.e., formulas for computing matrix elements.

### Simple Co Occurence Counts

The simplest approach is to define contexts as each word in an L-sized window. Matrix element for a word-context pair (w, c) is the number of times w appears in context c. This is the very basic (and very, very old) method for obtaining embeddings.

Context: Surrounding words in an L sized window

Matrix Element: N(w, c) number of time words w appears in c

The [old grumpy cat started playing] in my room
the context for cat is: [old grumpy cat started playing]
the matrix element is: 1

Interesting Question: Are context words at different distances equally important? If not, how can we modify co-occurrence counts?

HAL Model: https://link.springer.com/content/pdf/10.3758/BF03204766.pdf

### Positive Pointwise Mutual Information(PPMI)

Here contexts are defined as before, but the measure of the association between word and context is more clever: positive PMI (or PPMI for short). PPMI measure is widely regarded as state-of-the-art for pre-neural distributional-similarity models.

Context: Surrounding words in an L sized window

Matrix Element:

$$PPMI(w, c) = max(0, PMI(w, c))$$
$$PMI(w, c) = \log{\frac{P(w, c)}{P(w)P(c)}} = \log{\frac{N(w, c)|(w,c)|}{N(w)N(c)}}$$

### Latent Semantic Analysis

Latent Semantic Analysis (LSA) analyzes a collection of documents. While in the previous approaches contexts served only to get word vectors and were thrown away afterward, here we are also interested in context, or, in this case, document vectors. LSA is one of the simplest topic models: cosine similarity between document vectors can be used to measure similarity between documents.

The term "LSA" sometimes refers to a more general approach of applying SVD to a term-document matrix where the term-document elements can be computed in different ways (e.g., simple co-occurrence, tf-idf, or some other weighting).

context: document d (from a collection D)

Matix Element:
$$tf_idf(w, d, D) = tf(w, d)*idf(w, D)$$

Resources:

* http://lsa.colorado.edu/papers/JASIS.lsi.90.pdf
* https://en.wikipedia.org/wiki/Latent_semantic_analysis
* https://en.wikipedia.org/wiki/Tf%E2%80%93idf